In [1]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import as_strided
import warnings

# Tắt các cảnh báo (warnings) không cần thiết
warnings.filterwarnings('ignore', category=RuntimeWarning)

def calculate_technical_indicators(
    df: pd.DataFrame, 
    period: int = 20, 
    decay_factor: float = 0.95, 
    decay_lambda: float | None = None,
    sma_window: int = 20,
    bollinger_window: int = 20,
    rsi_window: int = 14,
    macd_fast: int = 12,
    macd_slow: int = 26,
    macd_signal: int = 9
) -> pd.DataFrame:
    """
    Tính toán các chỉ số kỹ thuật VWAP, WCP, ZLMA, FWMA, Decay
    SMA, Bollinger Bands, RSI, MACD.
    """
    data = df.copy()

    # 1. Weighted Close Price (WCP)
    data['WCP'] = (data['High'] + data['Low'] + 2 * data['Price']) / 4

    # 2. Volume Weighted Average Price (VWAP)
    data['WAP'] = (data['High'] + data['Low'] + data['Price']) / 3
    vwap_num = (data['WAP'] * data['Vol.']).rolling(window=period).sum()
    vwap_den = data['Vol.'].rolling(window=period).sum()
    data['VWAP'] = vwap_num / vwap_den

    # 3. Zero-Lag Moving Average (ZLMA)
    ema1 = data['Price'].ewm(span=period, adjust=False).mean()
    source = 2 * data['Price'] - ema1
    data['ZLMA'] = source.ewm(span=period, adjust=False).mean()

    # 4. Fibonacci's Weight Moving Average (FWMA)
    def fwma_fibonacci(prices: np.ndarray) -> float:
        n = len(prices)
        fib = [1, 1]
        for k in range(2, n):
            fib.append(fib[-1] + fib[-2])
        fib = np.array(fib)
        return np.dot(prices, fib) / np.sum(fib)

    if period >= 2:
        data['FWMA'] = data['Price'].rolling(window=period).apply(fwma_fibonacci, raw=True)
    else:
        data['FWMA'] = np.nan

    # 5. Decay - VWAP (exponential decay)
    if decay_lambda is None:
        if not (0.0 < decay_factor < 1.0):
            raise ValueError("decay_factor phải nằm trong (0, 1).")
        decay_lambda = -np.log(decay_factor)
    
    n = len(data)
    if len(data) >= period:
        price_vals = data['Price'].to_numpy(dtype=float)
        vol_vals = data['Vol.'].to_numpy(dtype=float)

        n_win = n - period + 1
        s_price = price_vals.strides[0]
        s_vol = vol_vals.strides[0]
        price_windows = as_strided(price_vals, shape=(n_win, period), strides=(s_price, s_price))
        vol_windows   = as_strided(vol_vals,   shape=(n_win, period), strides=(s_vol, s_vol))

        i = np.arange(period - 1, -1, -1, dtype=float)
        decay_weights = np.exp(-decay_lambda * i)
        numerator = np.sum(price_windows * vol_windows * decay_weights, axis=1, dtype=float)
        denominator = np.sum(vol_windows * decay_weights, axis=1, dtype=float)
        decay_result = np.divide(numerator, denominator, out=np.full_like(numerator, np.nan), where=denominator != 0)
        data['Decay'] = np.concatenate([np.full(period - 1, np.nan), decay_result])
    else:
        data['Decay'] = np.nan

    # 6. Simple Moving Averages (SMA)
    # Tính SMA cho Bollinger Bands và dùng chung
    data['SMA'] = data['Price'].rolling(window=sma_window).mean()

    # 7. Bollinger Bands
    # Dùng SMA đã tính
    if sma_window == bollinger_window:
        sma_bb = data['SMA']
    else:
        sma_bb = data['Price'].rolling(window=bollinger_window).mean()
        
    data['Bollinger_Std'] = data['Price'].rolling(window=bollinger_window).std()
    data['Bollinger_Upper'] = sma_bb + (data['Bollinger_Std'] * 2)
    data['Bollinger_Lower'] = sma_bb - (data['Bollinger_Std'] * 2)
    data = data.drop(columns=['Bollinger_Std'])

    # 8. Relative Strength Index (RSI) 
    delta = data['Price'].diff(1)
    
    # 9. Average Gain/Loss
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    
    # Sử dụng ewm (Wilder's smoothing)
    data['Avg_Gain'] = gain.ewm(com=rsi_window - 1, min_periods=rsi_window, adjust=False).mean()
    data['Avg_Loss'] = loss.ewm(com=rsi_window - 1, min_periods=rsi_window, adjust=False).mean()
    
    # Tính RSI
    rs = data['Avg_Gain'] / data['Avg_Loss']
    data['RSI'] = 100.0 - (100.0 / (1.0 + rs))
    
    # Xử lý trường hợp Avg_Loss = 0 (dẫn đến RS = Inf và RSI = 100)
    data['RSI'] = data['RSI'].replace([np.inf, -np.inf], np.nan) # Thay thế Inf (nếu có)
    data.loc[data['Avg_Loss'] == 0, 'RSI'] = 100 # Khi Avg_Loss = 0, RSI là 100

    # 10. Moving Average Convergence Divergence (MACD)
    ema_fast = data['Price'].ewm(span=macd_fast, adjust=False).mean()
    ema_slow = data['Price'].ewm(span=macd_slow, adjust=False).mean()
    
    data['MACD'] = ema_fast - ema_slow
    data['MACD_Signal'] = data['MACD'].ewm(span=macd_signal, adjust=False).mean()
    
    # Tính MACD Histogram (sự khác biệt giữa MACD và Signal line)
    data['MACD_Hist'] = data['MACD'] - data['MACD_Signal']

    return data


# --- BẮT ĐẦU THỰC THI ---
try:
    # 1. Đọc dữ liệu
    df = pd.read_csv('cleaned_stock_data.csv')

    # 2. Tiền xử lý
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Ticker', 'Date'])

    # 3. Áp dụng hàm tính toán
    df_with_indicators = df.groupby('Ticker', group_keys=False).apply(
        calculate_technical_indicators, 
        period=20,
        decay_factor=0.95
    )
    
    # 4. XUẤT RA TỆP CSV
    output_filename = 'stock_data_with_indicators.csv'
    df_with_indicators.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"Hoàn thành! Dữ liệu đã được xuất thành công ra tệp '{output_filename}'.")

except FileNotFoundError:
    print("Lỗi: Không tìm thấy tệp 'cleaned_stock_data.csv'.")
    print("Vui lòng đảm bảo tệp dữ liệu nằm cùng thư mục với file code.")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

C:\Users\ZenBook\AppData\Local\Temp\ipykernel_19556\2200899842.py:139: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_with_indicators = df.groupby('Ticker', group_keys=False).apply(


Hoàn thành! Dữ liệu đã được xuất thành công ra tệp 'stock_data_with_indicators.csv'.
